In [98]:
#Data Ingestion and Extraction
from pyspark.sql import SparkSession
jobDir = "tweets1.json"
tweets = spark.read.json([jobDir])
tweets.count() 

7992

In [99]:
tweets = tweets.select("text", \
                     "Category" )

tweets.printSchema()

root
 |-- text: string (nullable = true)
 |-- Category: string (nullable = true)



In [100]:
#Model Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words")

countVectors = CountVectorizer(inputCol="words", outputCol="features")



In [101]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, countVectors, label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(tweets)
dataset = pipelineFit.transform(tweets)
dataset.show(10,100)

+-------------------------------------------------+--------+-----------------------------------------------------------+----------------------------------------------------------------------------------------------+-----+
|                                             text|Category|                                                      words|                                                                                      features|label|
+-------------------------------------------------+--------+-----------------------------------------------------------+----------------------------------------------------------------------------------------------+-----+
|               والل عجب عشان كتاب انجليز صعب كلم |     POS|                  [والل, عجب, عشان, كتاب, انجليز, صعب, كلم]|                          (17072,[68,109,147,351,644,1456,6074],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|  2.0|
|               انه مفيد جدان انا اتعلم كثير اشيء |     POS|                  [انه, مفيد, جدان, انا, اتعلم, كثير

In [102]:
#Partition Training & Test sets
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 5654
Test Dataset Count: 2335


In [103]:
#Logistic Regression using Count Vector Features
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("text","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)


+------------------------------+--------+------------------------------+-----+----------+
|                          text|Category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|قل اذا هدد وتح ضغط بعت بنان...|     NEG|[0.9999951647125606,1.45473...|  0.0|       0.0|
|رييس تحرير عكاظ سعوديه راض ...|     NEG|[0.9930659029159837,0.00519...|  0.0|       0.0|
|اين دوله عربيه اعط فلسطين ط...|     NEG|[0.9891689833078323,0.01061...|  0.0|       0.0|
|يوم استقال غلبط بنان سعود ق...|     NEG|[0.9768467931718616,0.01140...|  0.0|       0.0|
|صار ريق جداد عالساح جماع بح...|     NEG|[0.9696483677997423,0.01513...|  0.0|       0.0|
|سعوديه صفيح هيب ساخ متفجر ق...|     NEG|[0.9679709021898679,0.01766...|  0.0|       0.0|
|تحدث جنون جماهير بوك ريفر س...|     POS|[0.9670284653611705,0.00372...|  2.0|       0.0|
|حقين سكران سكره رضو ضرايب م...|     NEG|[0.9655204386634646,0.02023...|  0.0|       0.0|
|ليش شركا 

In [117]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.2948332644602227

In [105]:
#Logistic Regression using TF-IDF Features
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
pipeline = Pipeline(stages=[regexTokenizer, hashingTF, idf, label_stringIdx])
pipelineFit = pipeline.fit(tweets)
dataset = pipelineFit.transform(tweets)
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("text","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                          text|Category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|قل اذا هدد وتح ضغط بعت بنان...|     NEG|[0.9999760363629308,1.68273...|  0.0|       0.0|
|اين دوله عربيه اعط فلسطين ط...|     NEG|[0.9992480937334544,3.48872...|  0.0|       0.0|
|ايا عمر فاروق عود جيوش فرس ...|     POS|[0.9854802398485728,0.00367...|  2.0|       0.0|
|سعوديه صفيح هيب ساخ متفجر ق...|     NEG|[0.9801727729784417,0.00880...|  0.0|       0.0|
|مسلماني اغل متحدث اعلامي ري...|     NEG|[0.9785276719739653,0.01913...|  0.0|       0.0|
|مسلماني اغل متحدث اعلامي ري...|     NEG|[0.9785276719739653,0.01913...|  0.0|       0.0|
|اهمال ادار مرافق عامه يود ل...|     NEG|[0.9735461069965777,0.01292...|  0.0|       0.0|
|تعلم ان حارس منتخب سور حال ...|     NEG|[0.9710639605171212,0.01478...|  0.0|       0.0|
|صفو حجاز 

In [106]:
#Cross-Validation
pipeline = Pipeline(stages=[regexTokenizer,countVectors, label_stringIdx])
pipelineFit = pipeline.fit(tweets)
dataset = pipelineFit.transform(tweets)
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
             .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)
cvModel = cv.fit(trainingData)

predictions = cvModel.transform(testData)
# Evaluate best model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.6811880616486291

In [107]:
#Naive Bayes
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1)
model = nb.fit(trainingData)
predictions = model.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("text","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                          text|Category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|قل اذا هدد وتح ضغط بعت بنان...|     NEG|[1.0,3.8926662561845564E-30...|  0.0|       0.0|
|اين دوله عربيه اعط فلسطين ط...|     NEG|[0.9999999999999618,3.81754...|  0.0|       0.0|
|يوم استقال غلبط بنان سعود ق...|     NEG|[0.999999999999921,7.914688...|  0.0|       0.0|
|رييس تحرير عكاظ سعوديه راض ...|     NEG|[0.9999999999995806,4.19351...|  0.0|       0.0|
|لم اكن اريد اكبر السن تنقلب...|     NEG|[0.9999999999670981,2.75367...|  0.0|       0.0|
|الله يستر قادم تكون استقال ...|     NEG|[0.9999999999017894,9.82041...|  0.0|       0.0|
|محمد بن سلمان يسير خطي صدام...|     NEG|[0.9999999987254311,1.14917...|  0.0|       0.0|
|الله يستر قادم تكون استقال ...|     NEG|[0.9999999959887278,4.01043...|  0.0|       0.0|
|تغييب دول

In [108]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.6642816494568803

In [109]:
#Random Forest
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
# Train model with Training Data
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("text","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 100, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                          text|Category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|نفاق انفصام امس استقبل مستش...|     NEG|[0.510664191235817,0.250437...|  0.0|       0.0|
|نفاق انفصام امس استقبل مستش...|     NEG|[0.510664191235817,0.250437...|  0.0|       0.0|
|نفاق انفصام امس استقبل مستش...|     NEG|[0.510664191235817,0.250437...|  0.0|       0.0|
|الي رجال اعمال اموال سعوديه...|     NEG|[0.5082835143911532,0.24858...|  0.0|       0.0|
|اظن ان افسد شيء فعل اصلاح ي...|     NEG|[0.507506887355017,0.255576...|  0.0|       0.0|
|سور لبن لاجء فيز جواز ضرايب...|     NEG|[0.5071252677764823,0.25109...|  0.0|       0.0|
|في زمن انقلاب قتل مسلم ملتح...|     NEG|[0.5061767079865583,0.25642...|  0.0|       0.0|
|الله يستر قادم تكون استقال ...|     NEG|[0.5046332567240381,0.25249...|  0.0|       0.0|
|الله يستر

In [110]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.2948332644602227

In [111]:
pl = predictions.select("label", "prediction")


In [112]:
pl.groupby('label').agg({'label': 'count'}).show()


+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|        1085|
|  1.0|         600|
|  2.0|         650|
+-----+------------+



In [118]:
pl.groupby('prediction').agg({'prediction': 'count'}).show()


+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|       0.0|             2335|
+----------+-----------------+



In [114]:
pl.filter(pl.label == pl.prediction).count() / pl.count()

0.46466809421841543

In [115]:
def accuracy_m(model): 
    predictions = model.transform(testData)
    pl = predictions.select("label", "prediction")
    acc = pl.filter(pl.label == pl.prediction).count() / pl.count()
    print("Model accuracy: %.3f%%" % (acc * 100)) 
accuracy_m(model = rfModel)


Model accuracy: 46.467%
